In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

# -------------------------------
# PARAMETERS
# -------------------------------
sim_PA_decay = 1  # whether we want decay in PA or not

# Decay parameters
gamma1 = 11         # exponential decay parameter for EA
gamma2 = 20         # exponential decay parameter for PA (when EA starts)
alpha = 0.015       # baseline (not used further)
alpha_d = 0.05      # height of sigmoid
mu_d = 0.3          # mean of sigmoid
sigma_d = 0.3       # width of sigmoid

# EA parameters
mu2 = 9             # effective drift
t_theta = 3.15      # time scale

# PA parameters
mu1 = 2           # drift
theta1 = 1.85     # bound

# Fixation and offset parameters
fix_mean = 0.3
o_EA = 0.07   # offset for EA
o_PA = 0      # offset for PA
o_fix = 0.2   # offset for fixation
t_cut = 0.3   # time to truncate (not used in simulation)

n_trials = int(20e3)  # number of trials

# -------------------------------
# SIMULATE EA PROCESS
# -------------------------------
dt = 1e-3
tt = np.arange(0, 3 * t_theta * 0.75 + dt, dt)  # time vector from 0 to 3*t_theta*0.75
n_t = len(tt)

theta2 = 1  # threshold for EA

# initialize t_2 as the default (last time point) for each trial
t_2 = np.full(n_trials, tt[-1])
tt_EA = tt.copy()

# Compute the decay function for EA:
# f_decay_EA = exp(-gamma1*t) + alpha_d/(1+exp(-(t-mu_d)/sigma_d^2))
f_decay_EA = np.exp(-gamma1 * tt) + alpha_d / (1 + np.exp(-(tt - mu_d) / (sigma_d ** 2)))

# Define the integrated decay function (tau) and its inverse
def f_integral(t):
    """Integral of the decay function."""
    return (1 / gamma1) * (1 - np.exp(-gamma1 * t)) + \
           alpha_d * (sigma_d ** 2) * np.log((1 + np.exp((t - mu_d) / (sigma_d ** 2))) /
                                             (1 + np.exp(-mu_d / (sigma_d ** 2))))

def tau(t):
    """Alias for the integral function."""
    return f_integral(t)

def tau_inv_scalar(yi):
    """Find x such that tau(x) == yi using a scalar root finder."""
    a = 0.0
    b = 10.0
    # Increase the upper bound until tau(b) is above yi.
    while tau(b) < yi:
        b *= 2
    sol = root_scalar(lambda x: tau(x) - yi, bracket=[a, b], method='bisect')
    return sol.root

# Vectorize the inverse tau function so we can apply it to an array.
tau_inv = np.vectorize(tau_inv_scalar)

# --- EA simulation: "normal" version (with decay) ---
for j in range(n_trials):
    mu_vec = mu2 * f_decay_EA  # drift scaled by the decay function
    dW = np.random.randn(n_t)
    dx = dW * np.sqrt(dt) * np.sqrt(f_decay_EA) + mu_vec * dt
    x = np.cumsum(dx)
    x = np.abs(x)
    crossing = np.where(x > theta2)[0]
    if crossing.size > 0:
        t_2[j] = tt[crossing[0]]

# --- EA simulation: "rescaled" version (without decay) ---
tau_2 = np.full(n_trials, tt[-1])
for j in range(n_trials):
    mu_vec = mu2 * np.ones(n_t)
    dW = np.random.randn(n_t)
    dx = dW * np.sqrt(dt) + mu_vec * dt
    x = np.cumsum(dx)
    x = np.abs(x)
    crossing = np.where(x > theta2)[0]
    if crossing.size > 0:
        tau_2[j] = tt[crossing[0]]

# Invert the integrated time transformation and rescale back by t_theta
t_3 = tau_inv(tau_2) / t_theta
t_4 = t_2 / t_theta

# -------------------------------
# SIMULATE PA PROCESS
# -------------------------------
# Fixation: draw from an exponential distribution and add offset.
# x_fix = np.random.exponential(scale=fix_mean, size=n_trials) + o_fix

# if sim_PA_decay == 0:
#     # --- PA process without decay: sample from an inverse Gaussian ---
#     m = theta1 / mu1  # desired mean
#     lam = theta1 ** 2
#     v = np.random.randn(n_trials)
#     y = v ** 2
#     x_pa = m + (m ** 2 * y) / (2 * lam) - (m / (2 * lam)) * np.sqrt(4 * m * lam * y + m ** 2 * y ** 2)
#     u = np.random.rand(n_trials)
#     x_pa = np.where(u <= m / (m + x_pa), x_pa, m ** 2 / x_pa)
#     x_PA = x_pa + o_PA - x_fix
# else:
#     # --- PA process with decay ---
#     dt_pa = 5e-4
#     tt_pa = np.arange(0, 4 + dt_pa, dt_pa)  # time vector from 0 to 4 seconds
#     n_t_pa = len(tt_pa)
    
#     x_PA = np.full(n_trials, np.inf)
#     # Precompute the decay function for PA:
#     f_decay_PA = np.exp(-gamma2 * tt_pa)
    
#     for j in range(n_trials):
#         f_decay_PA_2 = np.ones(n_t_pa)
#         t_EA_start = x_fix[j] + o_EA
#         t_EA_start_ind = int(np.ceil(t_EA_start / dt_pa))
#         t_end_ind = n_t_pa - t_EA_start_ind
#         if t_EA_start_ind < n_t_pa:
#             # Fix: remove the extra element by using :t_end_ind instead of :t_end_ind+1
#             f_decay_PA_2[t_EA_start_ind:] = f_decay_PA[:t_end_ind]
#         mu_vec = mu1 * f_decay_PA_2
#         dW = np.random.randn(n_t_pa)
#         dx = dW * np.sqrt(dt_pa) * np.sqrt(f_decay_PA_2) + mu_vec * dt_pa
#         x = np.cumsum(dx)
#         crossing = np.where(x > theta1)[0]
#         if crossing.size > 0:
#             x_PA[j] = tt_pa[crossing[0]]
    
#     x_PA = x_PA + o_PA - x_fix

# -------------------------------
# COMBINE EA AND PA RESPONSES
# -------------------------------
# x_EA = t_2 / t_theta + o_EA
# x_win = np.minimum(x_PA, x_EA)
